In [3]:
#Import Libraries
import pandas as pd
import numpy as np

In [17]:
#Load datasets
customer = pd.read_excel(r"C:\Users\grace\Downloads\Data Integreation\CustomersData.xlsx")
online = pd.read_csv(r"C:\Users\grace\Downloads\Data Integreation\Online_Sales.csv")
data = pd.read_csv(r"C:\Users\grace\Downloads\Data Integreation\ecom_data.csv",encoding="latin1")


In [18]:
# Inspect
print(customer.head())
print(online.head())
print(data.head())

   CustomerID Gender    Location  Tenure_Months
0       17850      M     Chicago             12
1       13047      M  California             43
2       12583      M     Chicago             33
3       13748      F  California             30
4       15100      M  California             49
   CustomerID  Transaction_ID Transaction_Date     Product_SKU  \
0       17850           16679         1/1/2019  GGOENEBJ079499   
1       17850           16680         1/1/2019  GGOENEBJ079499   
2       17850           16681         1/1/2019  GGOEGFKQ020399   
3       17850           16682         1/1/2019  GGOEGAAB010516   
4       17850           16682         1/1/2019  GGOEGBJL013999   

                                 Product_Description Product_Category  \
0  Nest Learning Thermostat 3rd Gen-USA - Stainle...         Nest-USA   
1  Nest Learning Thermostat 3rd Gen-USA - Stainle...         Nest-USA   
2              Google Laptop and Cell Phone Stickers           Office   
3  Google Men's 100% Co

In [27]:
# Checking Data Type
print(customer.info())
print(online.info())
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1468 entries, 0 to 1467
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CustomerID     1468 non-null   int64 
 1   Gender         1468 non-null   object
 2   Location       1468 non-null   object
 3   Tenure_Months  1468 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 46.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52924 entries, 0 to 52923
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           52924 non-null  int64  
 1   Transaction_ID       52924 non-null  int64  
 2   Transaction_Date     52924 non-null  object 
 3   Product_SKU          52924 non-null  object 
 4   Product_Description  52924 non-null  object 
 5   Product_Category     52924 non-null  object 
 6   Quantity             52924 non-null  int64  
 7   Avg_Price       

In [32]:
# Keep only rows where customer_id is not missing
data = data[data['CustomerID'].notna()].copy()

# Convert customer_id from float to int
data["CustomerID"] = data["CustomerID"].astype(int)

In [33]:
# Create total revenue column
data['total_revenue_ecom'] = data['Quantity'] * data['UnitPrice']

# Aggregate per customer
data_agg = data.groupby('CustomerID').agg(
    total_orders_ecom=('InvoiceNo', 'count'),
    total_quantity_ecom=('Quantity', 'sum'),
    total_revenue_ecom=('total_revenue_ecom', 'sum')
).reset_index()


In [45]:
# Create total spent column
online['total_spent']= online['Quantity']*online['Avg_Price']+online['Delivery_Charges']

In [46]:
# Aggregate per customer
sales_agg=online.groupby('CustomerID').agg(
    total_orders=('Transaction_ID','count'),
    total_quanity=('Quantity','sum'),
    total_spent_online=('total_spent','sum')).reset_index()

In [47]:
# Create average order value column
sales_agg['avg_order_value']=sales_agg['total_spent_online']/sales_agg['total_orders']

In [48]:
# Merge Customer + Online
customer_master= customer.merge(sales_agg, on='CustomerID',how='left')

# Merge Customer + Data
customer_master= customer_master.merge(data_agg, on='CustomerID',how='left')


In [49]:
print(customer_master.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1468 entries, 0 to 1467
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           1468 non-null   int64  
 1   Gender               1468 non-null   object 
 2   Location             1468 non-null   object 
 3   Tenure_Months        1468 non-null   int64  
 4   total_orders         1468 non-null   int64  
 5   total_quanity        1468 non-null   int64  
 6   total_spent_online   1468 non-null   float64
 7   avg_order_value      1468 non-null   float64
 8   total_orders_ecom    1468 non-null   int64  
 9   total_quantity_ecom  1468 non-null   int64  
 10  total_revenue_ecom   1468 non-null   int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 126.3+ KB
None


In [51]:
print(customer_master.shape)

(1468, 11)


In [52]:
print(customer_master.isnull().sum())

CustomerID             0
Gender                 0
Location               0
Tenure_Months          0
total_orders           0
total_quanity          0
total_spent_online     0
avg_order_value        0
total_orders_ecom      0
total_quantity_ecom    0
total_revenue_ecom     0
dtype: int64


In [53]:
# Export to Excel
customer_master.to_excel("Customer360.xlsx", index=False)

In [54]:
#To check the path
import os
print(os.getcwd())


C:\Users\grace\Python
